# Authors: Joel Sole Casale, Nathaniel Mitrani

In [1]:
# install the required libraries if not already installed
# this can be omitted if the libraries are already installed
!pip install addfips --quiet
!pip install tqdm --quiet

## 1) Data cleaning

In [2]:
import pandas as pd
import addfips
from tqdm import tqdm

# Concat all data from GunViolenceArchive 2014-2023
df_data = pd.concat([pd.read_csv(f'data/GunViolenceByYear/{i}.csv') for i in range(2014, 2024)])
df_data.reset_index(drop=True, inplace=True)

In [3]:
df_data.head()

,Incident ID,Incident Date,State,City Or County,Address,Victims Killed,Victims Injured,Suspects Killed,Suspects Injured,Suspects Arrested,Operations
0,271363,"December 29, 2014",Louisiana,New Orleans,Poydras and Bolivar,0,4,0,0,0,NaN
1,269679,"December 27, 2014",California,Los Angeles,8800 block of South Figueroa Street,1,3,0,0,0,NaN
2,270036,"December 27, 2014",California,Sacramento,4000 block of May Street,0,4,0,0,0,NaN
3,269167,"December 26, 2014",Illinois,East St. Louis,2500 block of Summit Avenue,1,3,0,0,0,NaN
4,268598,"December 24, 2014",Missouri,Saint Louis,18th and Pine,1,3,0,0,0,NaN


We remove the Operations column as it is full of NA. We also remove the Address column as it is not useful for our analysis.

In [4]:
df_data.drop(columns=["Operations","Address"], inplace=True)

In [5]:
df_data.drop_duplicates(inplace=True)

In [6]:
# Cast columns that are full of numbers to int
for col in df_data.columns:
    try:
        df_data[col] = df_data[col].astype(int)
    except:
        pass

# Cast date to datetime
df_data["Incident Date"] = pd.to_datetime(df_data["Incident Date"])

## 2) Data augmentation

We add the county and its corresponding FIPS code to each row

In [7]:
us_city_states = pd.read_csv('data/us_cities_states_counties.csv', sep='|')
us_city_states["State full"] = us_city_states["State full"].replace("Washington, D.C.", "District of Columbia")


# create a dict to speed up the search
state_city_dict = {
    state: us_city_states[us_city_states["State full"]==state]
    for state in us_city_states["State full"].unique()
}

df_data["County"] = None
for i, row in tqdm(df_data.iterrows(), total=df_data.shape[0]):
    cityOrCounty = row["City Or County"]
    state = row["State"]

    # remove anything in parentheses
    cityOrCounty = cityOrCounty.split("(")[0].strip()
    
    possibleCounties = state_city_dict[state][state_city_dict[state]["City"]==cityOrCounty]["County"]

    if len(possibleCounties) >= 1:
        df_data.at[i, "County"] = possibleCounties.iloc[0]
    else:
        # search by city alias
        possibleCounties = state_city_dict[state][state_city_dict[state]["City alias"]==cityOrCounty]["County"]

        if len(possibleCounties) >= 1:
            df_data.at[i, "County"] = possibleCounties.iloc[0]


100%|██████████| 4683/4683 [00:03<00:00, 1341.95it/s]


In [8]:
print(df_data[df_data["County"].isnull()][["State", "City Or County"]].drop_duplicates())

               State   City Or County
3           Illinois   East St. Louis
73             Texas      Flour Bluff
214        Tennessee   Lookout Valley
479          Florida       Allapattah
1499         Alabama          Brandon
1706         Florida      Brownsville
3314  South Carolina        Mccormick
3429        Illinois  Washington Park
3852          Kansas  Cowley (county)
3900      California      Willowbrook
4346      California           Florin


In [9]:
manual_counties = {
    ("Illinois", "East St. Louis"): "St. Clair",
    ("Texas", "Flour Bluff"): "Nueces",
    ("Tennessee", "Lookout Valley"): "Hamilton",
    ("Florida", "Allapattah"): "Miami-Dade",
    ("Alabama", "Brandon"): "Jefferson",
    ("Florida", "Brownsville"): "Miami-Dade",
    ("South Carolina", "Mccormick"): "Mccormick",
    ("Illinois", "Washington Park"): "St. Clair",
    ("Kansas", "Cowley (county)"): "Cowley",
    ("California", "Willowbrook"): "Los Angeles",
    ("California", "Florin"): "Sacramento"
}

df_data["County"] = df_data.apply(lambda row: manual_counties.get((row["State"], row["City Or County"]), row["County"]), axis=1)

print(f"Missing counties: {df_data['County'].isnull().sum()}")

Missing counties: 0


In [10]:
# add the FIPS code using the addfips library
af = addfips.AddFIPS()

df_data["FIPS"] = None

for i, row in tqdm(df_data.iterrows(), total=df_data.shape[0]):
    county = row["County"]
    state = row["State"]

    if county is None:
        continue

    if county[:3] == "ST ": # fix ST. to ST
        county = "ST. " + county[3:]

    fips = af.get_county_fips(county, state)
    df_data.at[i, "FIPS"] = fips

100%|██████████| 4683/4683 [00:00<00:00, 8554.56it/s]


In [11]:
print(df_data[df_data["FIPS"].isnull()][["State", "City Or County", "County"]])

        State City Or County             County
2109   Alaska         Palmer  MATANUSKA SUSITNA
4339  Indiana  Michigan City           LA PORTE


In [12]:
manual_fips = {
    ("Alaska", "Palmer"): "02170",
    ("Indiana", "Michigan City"): "18091"
}

df_data["FIPS"] = df_data.apply(lambda row: manual_fips.get((row["State"], row["City Or County"]), row["FIPS"]), axis=1)

print(f"Missing FIPS codes: {df_data['FIPS'].isnull().sum()}")

Missing FIPS codes: 0


We add the population of each county to each row

In [13]:
df_population = pd.read_csv('data/co-est2023-alldata.csv', encoding='latin-1')

# Rename the STATE column to State
df_population.rename(columns={'STNAME': 'State'}, inplace=True)

df_pop_per_state = df_population[df_population['COUNTY'] == 0]

# Add population per state
df_data = df_data.merge(df_pop_per_state[['State','POPESTIMATE2023']], on='State', how='left')
df_data.rename(columns={'POPESTIMATE2023': 'Population_per_state_2023'}, inplace=True)

In [14]:
df_pop_per_counties = df_population[df_population['COUNTY'] != 0].copy()

df_pop_per_counties["FIPS"] = df_pop_per_counties["STATE"].astype(str).str.zfill(2) + df_pop_per_counties["COUNTY"].astype(str).str.zfill(3)

# Add population per county
df_data = df_data.merge(df_pop_per_counties[['FIPS','POPESTIMATE2023']], on='FIPS', how='left')
df_data.rename(columns={'POPESTIMATE2023': 'Population_per_county_2023'}, inplace=True)

In [15]:
df_data.to_csv('data/gun_violence_processed.csv', index=False)